In [1]:
import selenium
from selenium import webdriver
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH = 'chromedriver.exe'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [2]:
wd.get('https://www.google.com')

In [3]:
wd.quit()

In [11]:
import os
import time
import requests
import io

In [5]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [15]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content
        
    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [22]:
def search_and_download(search_term:str,driver_path:str, target_path='./OneDrive/Desktop/imagescaping',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        with webdriver.Chrome(executable_path=driver_path) as wd:
            res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=3)
        
    for elem in res:
        persist_image(target_folder,elem)

In [23]:
from PIL import Image
import hashlib
search_terms = ['ice']
for search_term in search_terms:
    search_and_download(search_term=search_term, driver_path=DRIVER_PATH)

Found: 100 search results. Extracting links from 0:100
Found: 5 image links, done!
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/6/67/Ice_Block%2C_Canal_Park%2C_Duluth_%2832752478892%29.jpg - as ./OneDrive/Desktop/imagescaping\ice\c47e5249ce.jpg
SUCCESS - saved https://clearlakeiowa.com/wp-content/uploads/2019/11/ice-cubes-e1573665814265.jpg - as ./OneDrive/Desktop/imagescaping\ice\f23885f949.jpg
SUCCESS - saved https://cdn.britannica.com/s:900x675/09/162509-131-9D36AD0E/Ice-cubes-background.jpg - as ./OneDrive/Desktop/imagescaping\ice\002f31a501.jpg
SUCCESS - saved https://api.time.com/wp-content/uploads/2020/01/IceHack.jpg - as ./OneDrive/Desktop/imagescaping\ice\0f23edefce.jpg
SUCCESS - saved https://carverdental.com/wp-content/uploads/2019/05/chewing-ice_620.jpg - as ./OneDrive/Desktop/imagescaping\ice\ed9a83a9c7.jpg
